In [1]:
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.20.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.20.04, mixed mode, sharing)
  Starting server from /home/gerd/Documents/Python/venvs/h2o/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmppwnktu60
  JVM stdout: /tmp/tmppwnktu60/h2o_gerd_started_from_python.out
  JVM stderr: /tmp/tmppwnktu60/h2o_gerd_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Vienna
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.2
H2O_cluster_version_age:,1 month and 8 days
H2O_cluster_name:,H2O_from_python_gerd_cqle2e
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.867 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [5]:
data = h2o.import_file('http://h2o-public-test-data.s3.amazonaws.com/smalldata/airlines/allyears2k_headers.zip')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
allColumns = data.columns
allColumns.remove("ArrDelay")

specificColumns = data.columns
specificColumns.remove("UniqueCarrier")
specificColumns.remove("TailNum")
specificColumns.remove("TaxiIn")
specificColumns.remove("TaxiOut")
specificColumns.remove("ArrDelay")

targetColumn = "IsArrDelayed"

In [7]:
train, validation, test = data.split_frame(ratios=[0.8, 0.1], destination_frames=["train", "validation", "test"], seed=123)

In [17]:
gbm_overfitted = H2OGradientBoostingEstimator(model_id = 'overfitted-model', ntrees=1000, max_depth=10)

In [18]:
gbm_overfitted.train(specificColumns, targetColumn, train, validation_frame=validation)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [19]:
gbm_overfitted.model_performance(test)


ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.03399778035701919
RMSE: 0.18438487019552116
LogLoss: 0.18012550237254465
Mean Per-Class Error: 0.04007673120177491
AUC: 0.9929667150833195
AUCPR: 0.9924977856539606
Gini: 0.9859334301666389

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.24047887144064248: 


,,NO,YES,Error,Rate
0,NO,1867.0,118.0,0.0594,(118.0/1985.0)
1,YES,62.0,2404.0,0.0251,(62.0/2466.0)
2,Total,1929.0,2522.0,0.0404,(180.0/4451.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,2.404789e-01,0.963913,236.0
1,max f2,1.630652e-02,0.976391,333.0
2,max f0point5,9.751413e-01,0.971074,70.0
3,max accuracy,7.594787e-01,0.959784,156.0
4,max precision,9.999995e-01,0.996071,0.0
5,max recall,9.754678e-06,1.000000,398.0
6,max specificity,9.999995e-01,0.995970,0.0
7,max absolute_mcc,7.594787e-01,0.918777,156.0
8,max min_per_class_accuracy,7.594787e-01,0.958637,156.0
9,max mean_per_class_accuracy,7.594787e-01,0.959923,156.0



Gains/Lift Table: Avg response rate: 55.40 %, avg score: 55.76 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.073916,1.000000e+00,1.804947,1.804947,1.000000,1.000000e+00,1.000000,1.000000,0.133414,0.133414,80.494728,80.494728,0.133414
1,2,0.101550,1.000000e+00,1.804947,1.804947,1.000000,1.000000e+00,1.000000,1.000000,0.049878,0.183293,80.494728,80.494728,0.183293
2,3,0.150079,1.000000e+00,1.804947,1.804947,1.000000,1.000000e+00,1.000000,1.000000,0.087591,0.270884,80.494728,80.494728,0.270884
3,4,0.200180,1.000000e+00,1.796853,1.802922,0.995516,1.000000e+00,0.998878,1.000000,0.090024,0.360908,79.685335,80.292153,0.360405
4,5,0.300157,1.000000e+00,1.800891,1.802245,0.997753,1.000000e+00,0.998503,1.000000,0.180049,0.540957,80.089122,80.224527,0.539949
5,6,0.400135,9.999998e-01,1.800891,1.801907,0.997753,1.000000e+00,0.998316,1.000000,0.180049,0.721006,80.089122,80.190695,0.719494
6,7,0.500112,9.989898e-01,1.727882,1.787109,0.957303,9.998932e-01,0.990117,0.999979,0.172749,0.893755,72.788212,78.710863,0.882672
7,8,0.600090,5.289637e-03,0.981567,1.652902,0.543820,5.747554e-01,0.915762,0.929135,0.098135,0.991890,-1.843316,65.290193,0.878540
8,9,0.700067,8.209368e-07,0.081121,1.428434,0.044944,4.110907e-04,0.791399,0.796502,0.008110,1.000000,-91.887877,42.843389,0.672544
9,10,0.800045,1.138787e-08,0.000000,1.249930,0.000000,1.778116e-07,0.692502,0.696968,0.000000,1.000000,-100.000000,24.992980,0.448363


In [20]:
"""
What the early stopping parameters do:

Average the error (in regards to the used stopping_metric) over the last n scoring rounds (n = stopping_rounds)
Compare the value to the score of the previous n scoring rounds (n = stopping rounds)
If the model has not gotten better by the value specified in stopping_tolerance, it stops learning
"""

gbm_early_stopping = H2OGradientBoostingEstimator(model_id = 'early-stopping-model', stopping_metric='logloss', 
                                                  stopping_tolerance=0.001 ,stopping_rounds=4, ntrees=1000, max_depth=10)

In [21]:
gbm_early_stopping.train(specificColumns, targetColumn, train, validation_frame=validation)

/home/gerd/Documents/Python/venvs/h2o/lib/python3.8/site-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: early stopping is enabled but neither score_tree_interval or score_each_iteration are defined. Early stopping will not be reproducible!
  warnings.warn(mesg["message"], RuntimeWarning)


gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [22]:
gbm_early_stopping.model_performance(test)


ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.032022613103102095
RMSE: 0.1789486325823757
LogLoss: 0.1457851732339194
Mean Per-Class Error: 0.03933281035176639
AUC: 0.9938632403202445
AUCPR: 0.9940786602134314
Gini: 0.9877264806404891

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.29531116962456505: 


,,NO,YES,Error,Rate
0,NO,1872.0,113.0,0.0569,(113.0/1985.0)
1,YES,57.0,2409.0,0.0231,(57.0/2466.0)
2,Total,1929.0,2522.0,0.0382,(170.0/4451.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.295311,0.965918,240.0
1,max f2,0.054687,0.976998,318.0
2,max f0point5,0.944945,0.972386,86.0
3,max accuracy,0.431044,0.961806,212.0
4,max precision,0.999929,0.998010,2.0
5,max recall,0.000024,1.000000,398.0
6,max specificity,0.999999,0.997985,0.0
7,max absolute_mcc,0.295311,0.922798,240.0
8,max min_per_class_accuracy,0.708757,0.958186,170.0
9,max mean_per_class_accuracy,0.431044,0.960667,212.0



Gains/Lift Table: Avg response rate: 55.40 %, avg score: 55.75 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.013480,1.000000e+00,1.804947,1.804947,1.000000,1.000000e+00,1.000000,1.000000,0.024331,0.024331,80.494728,80.494728,0.024331
1,2,0.020220,1.000000e+00,1.804947,1.804947,1.000000,1.000000e+00,1.000000,1.000000,0.012165,0.036496,80.494728,80.494728,0.036496
2,3,0.030106,1.000000e+00,1.804947,1.804947,1.000000,1.000000e+00,1.000000,1.000000,0.017843,0.054339,80.494728,80.494728,0.054339
3,4,0.040216,1.000000e+00,1.804947,1.804947,1.000000,1.000000e+00,1.000000,1.000000,0.018248,0.072587,80.494728,80.494728,0.072587
4,5,0.050101,1.000000e+00,1.804947,1.804947,1.000000,1.000000e+00,1.000000,1.000000,0.017843,0.090430,80.494728,80.494728,0.090430
5,6,0.100202,1.000000e+00,1.804947,1.804947,1.000000,1.000000e+00,1.000000,1.000000,0.090430,0.180860,80.494728,80.494728,0.180860
6,7,0.150079,1.000000e+00,1.804947,1.804947,1.000000,1.000000e+00,1.000000,1.000000,0.090024,0.270884,80.494728,80.494728,0.270884
7,8,0.200180,1.000000e+00,1.804947,1.804947,1.000000,1.000000e+00,1.000000,1.000000,0.090430,0.361314,80.494728,80.494728,0.361314
8,9,0.300157,9.999999e-01,1.796835,1.802245,0.995506,1.000000e+00,0.998503,1.000000,0.179643,0.540957,79.683516,80.224527,0.539949
9,10,0.400135,9.999958e-01,1.800891,1.801907,0.997753,9.999990e-01,0.998316,1.000000,0.180049,0.721006,80.089122,80.190695,0.719494


In [23]:
h2o.cluster().shutdown()

H2O session _sid_abbf closed.
